In [148]:
import os
import math
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torch.optim as optim

from sklearn import preprocessing as prep

In [29]:
class sampleDataset(Dataset):
    def __init__(self, csv_file, array_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.array_dir = array_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, index):
        array_path = os.path.join(self.array_dir, self.annotations.iloc[index, 0]+".npy")
        array = np.load(array_path)
        array = array.astype('float32')
        y_label = annotations.iloc[index,1]

        if self.transform:
            array = self.transform(array)
        
        return array, y_label

In [19]:
csv_path = "/home/richard/labrotation/processed_sample_data/human_origins_labels.csv"
array_path = "//home/richard/labrotation/processed_sample_data/arrays"
index = 0

In [20]:
annotations = pd.read_csv(csv_path)
array_dir = array_path

In [30]:
dataset = sampleDataset(csv_file=csv_path, array_dir=array_dir)
data_loader = DataLoader(dataset=dataset, batch_size=5, shuffle=True)

In [32]:
test_batch = next(iter(data_loader))
test_array = test_batch[0]
test_array.shape
test_array.dtype

torch.float32

In [34]:
'''
expected input: Tensor of dimensions N*C*in_features
output: Tensor of dimensions N*out_sets*⌈(in_features/m)⌉
N: batch size
C: number of channels (e.g. 4 for one-hot-encoding of SNPs)
in_features: number of input features (e.g. SNP positions)
out_sets: number of output sets (new channels)
m: how many in_features to group together
kernel_size: kernel of flat tensor: m*C
padding: should we padd at the end of the dataframe if in_features%m !=0? 
'''
class LocallyConnectedLayer(torch.nn.Module):
    def __init__(self, in_features, m, C=4, padding=True, bias=False, out_sets=4):
        super().__init__()
        self.in_features = in_features
        self.C = C
        self.m = m
        self.kernel_size = m*C
        self.padding = (m-(in_features%m))%m*C if padding else 0
        self.weight = nn.Parameter(torch.randn(1,self.kernel_size, out_sets))
        self.bias = nn.Parameter(torch.randn(1,out_sets)) if bias else None # with batchnorm we do not need bias
    
    def forward(self, x):
        x = x.transpose(-1,-2) # we need to transpose first to ensure that the channel values of one in_feature are next to each other after flattening
        x = x.flatten(1) # dim(N,in_features*C)
        x = F.pad(x,(0,self.padding))
        x = x.unfold(-1, size=self.kernel_size, step=self.kernel_size)
        x = torch.matmul(x,self.weight)
        if self.bias is not None:
            x = x+self.bias
        x = x = x.transpose(-1,-2) # transpose back to have the more convenient dimension order
        return x


In [4]:
class LCBlock(nn.Module):
    def __init__(self, in_features, m, out_sets=4, p=0.0):
        super().__init__()
        self.bn = nn.BatchNorm1d(out_sets)
        self.silu = nn.SiLU()
        self.drop = nn.Dropout(p)
        self.LCL1 = LocallyConnectedLayer(in_features, m=m, padding=True, out_sets=out_sets)
        self.LCL2 = LocallyConnectedLayer(in_features=math.ceil(in_features/m),m=m, padding=True, out_sets=out_sets)
        self.identity_downsample = nn.Linear(in_features, out_features=math.ceil(in_features/m**2)) if m!=1 else None

    def forward(self, x):
        identity = x

        x = self.bn(x)
        x = self.silu(x)
        x = self.LCL1(x)
        x = self.bn(x)
        x = self.silu(x)
        x = self.drop(x)
        x = self.LCL2(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x = x+identity
        return x

In [5]:
'''
expected input: flat tensor of shape N*in_features
expected output: flat tensor of shape N*out_features
N: batch size
'''
class FCBlock(nn.Module):
    def __init__(self, in_features, out_features, p=0.5):
        super().__init__()
        self.bn1 = nn.BatchNorm1d(in_features)
        self.bn2 = nn.BatchNorm1d(out_features)
        self.silu = nn.SiLU()
        self.drop = nn.Dropout(p)
        self.FCL1 = nn.Linear(in_features=in_features, out_features=out_features)
        self.FCL2 = nn.Linear(in_features=out_features, out_features=out_features)
        self.identity_downsample = nn.Linear(in_features, out_features=out_features) if in_features != out_features else None

    def forward(self, x):
        identity = x

        x = self.bn1(x)
        x = self.silu(x)
        x = self.FCL1(x)
        x = self.bn2(x)
        x = self.silu(x)
        x = self.drop(x)
        x = self.FCL2(x)

        if self.identity_downsample is not None:
            identity = self.identity_downsample(identity)

        x = x+identity
        return x

In [122]:
class GLN(nn.Module):
    def __init__(self, in_features, num_classes, num_residual_blocks=2, m1=2, m2=2, C=4, num_predictor_blocks=4):
        super().__init__()
        self.m1 = m1
        self.m2 = m2
        self.num_residual_blocks = num_residual_blocks
        self.num_predictor_blocks = num_residual_blocks
        self.LCL0 = LocallyConnectedLayer(in_features, m=m1)
        Output1 = math.ceil(in_features/m1)
        self.LCLayers = self.make_LCLayers(Output1)
        Output2 = math.ceil(Output1/(2*m2)**num_residual_blocks)*C # we flatten after the last block TO DO: IMPLEMENT ENVIRONMENT CONCATENATION
        self.FCLayers = self.make_predictorLayers(in_features=Output2)
        self.bn = nn.BatchNorm1d(256)
        self.silu = nn.SiLU()
        self.drop = nn.Dropout(p=0.5)
        self.Linear = nn.Linear(256,num_classes)
        

    def make_LCLayers(self, in_features):
        layers = []
        for block in range(self.num_residual_blocks):
            layers.append(LCBlock(in_features=in_features, m=self.m2))
            in_features = math.ceil(in_features/self.m2**2)
        return nn.Sequential(*layers)

    def make_predictorLayers(self, in_features):
        layers = []
        layers.append(FCBlock(in_features=in_features, out_features=256))
        for block in range(self.num_predictor_blocks):
            layers.append(FCBlock(in_features=256, out_features=256))
        return nn.Sequential(*layers)

    
    def forward(self,x):
        x = self.LCL0(x)
        x = self.LCLayers(x)
        x = x.flatten(1)
        x = self.FCLayers(x)
        x = self.bn(x)
        x = self.silu(x)
        x = self.drop(x)
        x = self.Linear(x)
        return x



In [141]:
# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device(type='cpu')

In [ ]:
# Hyperparameters
learning_rate = 1e-4
batch_size = 64


In [147]:
model = GLN(in_features=1000, num_classes=6,num_residual_blocks=2)
m = nn.LogSoftmax(dim=1)
result = model(test_array)
print(result, m(result))

tensor([[ 0.4346,  0.3174, -0.1007,  0.1731, -0.2837, -0.0323],
        [-0.6984,  0.1843,  0.0367, -0.2776, -0.6882,  0.1491],
        [ 0.0873,  0.3154, -0.7168, -0.5768,  0.5376, -0.0886],
        [ 0.1284,  0.3873,  0.0815, -0.2556,  0.2574,  0.5975],
        [-0.1336, -0.1177,  0.4921, -0.0596,  0.1685, -0.5919]],
       grad_fn=<AddmmBackward0>) tensor([[-1.4723, -1.5895, -2.0076, -1.7338, -2.1906, -1.9392],
        [-2.3389, -1.4562, -1.6039, -1.9182, -2.3288, -1.4915],
        [-1.7274, -1.4993, -2.5315, -2.3914, -1.2771, -1.9032],
        [-1.8972, -1.6383, -1.9441, -2.2811, -1.7682, -1.4281],
        [-1.9384, -1.9225, -1.3127, -1.8644, -1.6363, -2.3967]],
       grad_fn=<LogSoftmaxBackward0>)


In [132]:
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [156]:
classes = annotations['Origin'].unique()
le = prep.LabelEncoder()
le.fit(classes)
le.transform(test_batch[1])

array([1, 2, 2, 2, 3])

In [157]:
test_batch[1]

('Eastern_Asia',
 'Europe',
 'Europe',
 'Europe',
 'Latin_America_and_the_Caribbean')

In [158]:
classes

array(['Sub-Saharan_Africa', 'Europe', 'Asia', 'Middle_East',
       'Latin_America_and_the_Caribbean', 'Eastern_Asia'], dtype=object)

In [143]:
for epoch in range(2):
    
    running_loss = 0.0
    for batch_idx, (data,targets) in enumerate(data_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)

        
